# Setup

## Libraries

In [1]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'}

# Web Scrape

## Creating a function that extracts the posts' creation date, comment, and user detail

In [3]:
# Function
def get_comment_and_date(tag, page):
    
    url = f'https://www.speakev.com/threads/{tag}/page-{page}'
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    ######################
    ## Discussion title ##
    ######################
    
    title = soup.title.text
    dis_title.append(title)
    
    ##############################
    ## Extract date and comment ##
    ##############################
    message_cell = soup.find_all('div', {'class':'js-quickEditTarget message-cell-content-wrapper'})
    for item in message_cell:
        
        info1 = {
        'comment date' : item.find('time', {'class':'u-dt'})['datetime'],
        'comment' : item.find('div', {'class':'bbWrapper'}).text, 
        }
        
        list1.append(info1)
    
    #########################
    ## Extract user detail ##
    #########################
    
    message_userDetails = soup.find_all('div', {'class':'message-userDetails'})
    
    for item in message_userDetails:
    
        # User link
        link = 'https://www.speakev.com' + item.find('a', {'class': 'username'})['href']  

        # User profile url
        url_user = link
        r_user = requests.get(url_user, headers=headers)
        soup_user = BeautifulSoup(r_user.text, 'html.parser')
        
        # Number with tags (replies, discussion created, points)
        num_tag = soup_user.find_all('a', {'class':'fauxBlockLink-linkRow u-concealed'})
        result1 = re.findall(r'\n(.*?)\n</a>', str(num_tag))
        
        # Reaction score
        r_score = soup_user.find_all('dl', {'class': 'pairs pairs--rows'})
        result2 = re.findall(r'\n(.*?)\n</dd>\n</dl>', str(r_score))
        
        # User name
        name = soup_user.find_all('span', {'class': 'username'})
        result3 = re.findall(r'\>(.*?)</span>', str(name))
        
        info2 = {
        'date joined' : item.find('time', {'class': 'u-dt'})['datetime'],
        'replies' : result1[0],
        'discussion' : result1[1],
        'points' : result1[2],
        'reaction score' : result2[0], 
        'username' : result3[0],    
        }
        
        list2.append(info2)
    
    return

# Fiat discussion 1

In [4]:
# Extract relevant data using the function for fiat dicussion 1
tag = 'la-prima-first-edition-of-2021-500e.155648'

# Info list
list1 = []
list2 = []
dis_title = []

for x in range(1,3):
    get_comment_and_date(tag, x)

In [5]:
title = dis_title[0]
title

"'La Prima' first edition of 2021 500e | Speak EV - Electric Car Forums"

## User detail

In [8]:
df1_userinfo = pd.DataFrame(list2)

## Date created and comment

In [9]:
df1_post = pd.DataFrame(list1)

# Create a column identifying the author of each comment
df1_post['author'] = df1_userinfo['username']

## Likes

In [ ]:
# Class location
message_cell = soup.find_all('div', {'class':'js-quickEditTarget message-cell-content-wrapper'})
# Likes
likes = []

for item in message_cell:
   
    # likes
    for reaction in item('a', class_='reactionsBar-link', href=True):
        url_reaction = reaction['href']
        link = 'https://www.speakev.com' + url_reaction
        
        print(link)